<a href="https://colab.research.google.com/github/jhaashwaniii/NLP/blob/main/Glove_svm_english_hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import re
from pymongo import MongoClient
from collections import defaultdict
from nltk.corpus import wordnet as wn
import nltk
from nltk import word_tokenize
from nltk.util import ngrams as ngrams_creator
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import pickle
nltk.download('vader_lexicon')
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# train_path = "/content/drive/MyDrive/NLP/final_processed.csv"
# test_path = "/content/drive/MyDrive/NLP/val_processed.csv"
train_path = "/data/final_processed.csv"
test_path = "/data/val_processed.csv"
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(test_path)

In [ ]:
# trainX = df_train['sentence']
trainY = df_train["sentiment"]

In [ ]:
data = pd.concat([df_train,df_val]).reset_index()
data=data[["sentence","sentiment","positive_emoji","negative_emoji","neutral_emoji"]]

In [ ]:
data.shape

(16961, 5)

##**preprocess**


In [ ]:

def handle_short_forms(w):
    if w == 'h':
        return 'hai'
    elif w == 'n':
        return 'na'
    elif w == 'da':
        return 'the'
    elif w == 'wid':
        return 'with'
    elif w == 'pr':
        return 'par'
    elif w == 'mattt':
        return 'mat'
    elif w == 'vo':
        return 'woh'
    elif w == 'ki':
        return 'kee'
    elif w == 'ap':
        return 'aap'
    elif w == 'bs':
        return 'bas'
    elif w == 'goood':
        return 'very good'
    elif w == 'tera':
        return 'teraa'
    elif w == 'cnfsn':
        return 'confusion'
    elif w == 'ka':
        return 'kaa'
    elif w == 'rkhi':
        return 'rakhi'
    elif w == 'thts':
        return 'thats'
    elif w == 'cald':
        return 'called'
    elif w == 'tabhe':
        return 'tabhi'
    elif w == 'pta':
        return 'pata'
    elif w == 'b':
        return 'bhi'
    elif w == 'nai':
        return 'nahi'
    elif w == 'f':
        return 'of'
    elif w == 'd':
        return 'the'
    else:
        return w
    
'''
Translate word . 
'''
def translate(word):
    return translator.translate(word,src='hi' , dest='en').text

'''
Self defined contractions
''' 
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "i'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

'''
Handling short forms and contractions in the sentences 
'''
long_form_dict = load_dict_contractions()
def expand_sent(sentence):
    final_sent =""
    res = " ".join(long_form_dict.get(ele, ele) for ele in sentence.split()) 
    for word in res.split():
        final_sent += (handle_short_forms(word))+ " "
    return final_sent

##work

In [ ]:
l=[]
for sent in data.sentence:
  l.append(expand_sent(sent))
print(l[2:5])

['television media congress ke liye nhi hai Ye toh aapko pata chal hi gya hoga Achha hoga kee Congress ke ', 'All India nrc lagu kare w Kashmir se dhara 370ko khatam kare ham Indian ko apse yahi umid hai ', 'Pagal hai kya They real issues Mandir important Hindu khatre mei jo hai ']


##Slang word Processing



In [ ]:
slang_list = pd.read_csv("/data/Hinglish_Profanity_List.csv",sep=",",header=None,encoding= 'unicode_escape')
slang_list.columns=['hindi_word','meaning','rating']

In [ ]:
slang_list

,hindi_word,meaning,rating
0,badir,idiot,1
1,badirchand,idiot,1
2,bakland,idiot,1
3,bhadva,pimp,2
4,bhootnika,son of a witch,3
...,...,...,...
204,vahiyaat,disgusting,4
205,jihadi,terrorist,4
206,atankvadi,terrorist,4
207,atankwadi,terrorist,4


In [ ]:
def find_slang(sentence):
    slang_exists = False
    total_rating = 0
    for word in sentence.split():
        if not slang_list[slang_list['hindi_word']==word].empty:
            try:
                total_rating += slang_list.iloc[slang_list.index[slang_list['hindi_word'] == word].tolist()[0]]['rating']
            except:
                total_rating +=0
            slang_exists = True
    try:
        total_rating = int(total_rating)
    except:
        total_rating = 0
    return slang_exists,total_rating

In [ ]:
data['slang_exists']= data['sentence'].apply(find_slang)

In [ ]:
data[['slang_existance', 'slang_rating']] = pd.DataFrame(data['slang_exists'].tolist(), index=data.index) 
data =data.drop(columns=['slang_exists'])

In [ ]:
data["slang_existance"]==True

0        False
1        False
2        False
3        False
4        False
         ...  
16956    False
16957    False
16958    False
16959    False
16960    False
Name: slang_existance, Length: 16961, dtype: bool

In [ ]:
for i,j,k in zip(data['slang_existance'],data['sentence'],data['slang_rating']):
  if(i==True):
    print(j,"    ",k)

Baih tere itjey Kya jalti hai Pakistan se chutiye khelne de na unhe bc tu Indian bhi nahi h      10
tani Abe haattt tum flop logo ki aukaad hi Kiya h chutiya log ja aapijaan ki chaplusi kar bad      5
Phir se lut chalu ho gaya Bjp walo tumhe kuch chut mil raha hai ya nahi Milna chahiye q ki      8
jan Sharam haya ki qimat lagwai hai liay kute ko ye seat miliy Har haqomat k talway chatny wala hai ye      2
Haramzade hamare tukdo pe palta hai hamein aakh dikhata hai kutte tu pagal ho gaya      2
Jo bjp ke saath nahi woh atankwadi h aour jo hindu RSS ke saath nahi woh deshdrohi hai aaj kal b      4
Tum chutiya hoo gandu cogress harne wali sub ko pata tha itne buti tarah haarnegi nahi pata hai lavde      14
Stuffs Salman continue fuck chindiwaali filmfare So best luck mote bhains      1
Sir ye oonki aadat may shumaar hbycot se media deshwasion ko bhi pata chalegaki gandi naali k      3
Samay Samay ki baat hi rathee ab ModiIsNoDivider timemagazine ispr kch bolne ki himmat bc      10
O kutte

##Emoji Sentiment Analysis


In [ ]:
emoji_list = pd.read_csv('/data/Emoji_Sentiment_Data.csv',sep=",")

In [ ]:
emoji_list

,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block
0,😂,0x1f602,14622,0.805101,3614,4163,6845,FACE WITH TEARS OF JOY,Emoticons
1,❤,0x2764,8050,0.746943,355,1334,6361,HEAVY BLACK HEART,Dingbats
2,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,0x1f60d,6359,0.765292,329,1390,4640,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,0x1f62d,5526,0.803352,2412,1218,1896,LOUDLY CRYING FACE,Emoticons
...,...,...,...,...,...,...,...,...,...
964,➛,0x279b,1,0.011628,0,1,0,DRAFTING POINT RIGHTWARDS ARROW,Dingbats
965,♝,0x265d,1,0.280000,0,1,0,BLACK CHESS BISHOP,Miscellaneous Symbols
966,❋,0x274b,1,0.888889,0,1,0,HEAVY EIGHT TEARDROP-SPOKED PROPELLER ASTERISK,Dingbats
967,✆,0x2706,1,0.557252,0,1,0,TELEPHONE LOCATION SIGN,Dingbats


In [ ]:
def find_emoji(sentence):
    positive = 0
    negative = 0
    neutral  = 0
    sentiment =[]
    for word in sentence.split():
        if not emoji_list[emoji_list['Emoji']==word].empty:
            positive += emoji_list.iloc[emoji_list.index[emoji_list['Emoji'] == word].tolist()[0]]['Positive']
            negative += emoji_list.iloc[emoji_list.index[emoji_list['Emoji'] == word].tolist()[0]]['Negative']
            neutral += emoji_list.iloc[emoji_list.index[emoji_list['Emoji'] == word].tolist()[0]]['Neutral']
    return positive,negative,neutral


In [ ]:
data['emoji_sentiment'] = data['sentence'].apply(find_emoji)

## Upar wala part karana hai

##Calculating the intensity of sentiment in a sentence using NLTK

In [ ]:
sid=SentimentIntensityAnalyzer()
neg_score=[]
pos_score=[]
neu_score=[]
for sent in data.sentence:
  ss = sid.polarity_scores(sent)
  neg_score.append(ss['neg'])
  pos_score.append(ss['pos'])
  neu_score.append(ss['neu'])


In [ ]:
neg_score

[0.133,
 0.163,
 0.0,
 0.0,
 0.0,
 0.0,
 0.235,
 0.0,
 0.0,
 0.267,
 0.0,
 0.12,
 0.0,
 0.245,
 0.0,
 0.0,
 0.0,
 0.269,
 0.0,
 0.703,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.186,
 0.242,
 0.0,
 0.0,
 0.167,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.268,
 0.0,
 0.598,
 0.0,
 0.0,
 0.0,
 0.0,
 0.556,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.126,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.194,
 0.202,
 0.0,
 0.0,
 0.0,
 0.13,
 0.0,
 0.0,
 0.115,
 0.0,
 0.0,
 0.194,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.225,
 0.0,
 0.0,
 0.0,
 0.173,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.198,
 0.0,
 0.0,
 0.167,
 0.0,
 0.0,
 0.0,
 0.0,
 0.075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.106,
 0.0,
 0.35,
 0.0,
 0.224,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.059,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.201,
 0.0,
 0.0,
 0.0,
 0.112,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.15,
 0.37,
 0.0,
 0.403,
 0.204,
 0.363,
 0.0,
 0.0,
 0.

In [ ]:
data['neg_score']= neg_score
data['pos_score']= pos_score
data['neu_score']= neu_score

In [ ]:
data.tail(10)

,sentence,sentiment,positive_emoji,negative_emoji,neutral_emoji,slang_existance,slang_rating,neg_score,pos_score,neu_score
16951,digital Kya bakchodi hai flop actor ko Batman ...,neutral,0,0,0,False,0,0.130,0.000,0.870
16952,Me Modi ji aap aam kaise khaate hain,neutral,0,0,0,False,0,0.000,0.000,1.000
16953,Kya ho gaya aajkal political party ko Kya BJP ...,neutral,0,0,0,False,0,0.000,0.172,0.828
16954,Rajya sabha se apko mauka milna chahiye good t...,positive,0,0,0,False,0,0.000,0.209,0.791
16955,Krishna Rahul kanwal ko rajdeep ki hwa lag gyi...,negative,0,0,0,False,0,0.072,0.419,0.509
16956,Kisi Ko koi Khushi nahi Nayi sarkaar aati naye...,negative,0,0,0,False,0,0.000,0.000,1.000
16957,Music life Thank Chhote Ustad ( Salman Ali ) p...,neutral,0,0,0,False,0,0.000,0.378,0.622
16958,gilmour Hmmmm really Sam outlaw NOT something ...,neutral,0,0,0,False,0,0.133,0.397,0.470
16959,Ab gala faad ke nahi chillana chowkidaar chor ...,negative,0,0,0,False,0,0.000,0.000,1.000
16960,O lerki Allah SWT ko behas may ku lati ho 1 ch...,neutral,0,0,0,False,0,0.000,0.000,1.000


##data Prep

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# removing unwanted patterns from the data
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Dividing training the test dataset

train = data[:12000]
test = data[12000:]
train

In [ ]:
##fetching the sentence/comments from the training dataset and cleaning it

train_corpus = []

for i in range(0, len(train)):
  review = re.sub('[^a-zA-Z]', ' ', train['sentence'][i])
  review = review.lower()
  review = review.split()
  
  ps = PorterStemmer()
  
  # stemming
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  
  # joining them back with space
  review = ' '.join(review)
  train_corpus.append(review)

In [ ]:
##fetching the sentence/comments from the test dataset and cleaning it
test_corpus = []

for i in range(len(train), len(data)):
  review = re.sub('[^a-zA-Z]', ' ', test['sentence'][i])
  review = review.lower()
  review = review.split()
  
  ps = PorterStemmer()
  
  # stemming
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  
  # joining them back with space
  review = ' '.join(review)
  test_corpus.append(review)

In [ ]:
#Conversion of snetiment into numerical sentiment

train =train.replace("positive",1)
train =train.replace("negative",2)
train = train.replace("neutral",0)

In [ ]:
test =test.replace("positive",1)
test =test.replace("negative",2)
test = test.replace("neutral",0)

In [ ]:
# '''
# bag of words for the training data
# '''

# bag_of_words_vectorizer = CountVectorizer(max_features=1000)
# X = bag_of_words_vectorizer.fit_transform(train_corpus)
# words_counts = X.toarray()
# tfidf_transformer = TfidfTransformer()
# tfidf = tfidf_transformer.fit_transform(words_counts)

y_train,y_val = train.iloc[:, 1],test.iloc[:, 1]
print(len(y_train),len(y_val))

12000 4961


In [ ]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

data_tok = [tokenizer.tokenize(d.lower()) for d in train_corpus]

import gensim.downloader as api
model = api.load('glove-twitter-50')

import numpy as np
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. Just take an 
    average of vectors for all tokens in the phrase with some weights.
    """
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    # 1. lowercase phrase
    phrase = phrase.lower()
    
    # 2. tokenize phrase
    phrase_tokens = tokenizer.tokenize(phrase)
    
    # 3. average word vectors for all words in tokenized phrase, skip words that are not in model's vocabulary
    divisor = 0
    for word in phrase_tokens:
        if word in model.vocab:
            divisor += 1
            vector = vector + model.get_vector(word)
    
    if divisor != 0: vector /= divisor
    
    return vector

'''
Embeddings for Training Corpus
'''
vector_matrix_x_train = list(map(get_phrase_embedding, train_corpus))

'''
Embeddings for Training Corpus
'''
vector_matrix_x_test = list(map(get_phrase_embedding, test_corpus))

numerical_features_t = train[[ 'positive_emoji', 'negative_emoji', 'neutral_emoji', 'slang_rating','neg_score','pos_score','neu_score']]
print(numerical_features_t.values.tolist())

[[0.0, 0.0, 0.0, 0.0, 0.133, 0.0, 0.867], [1896.0, 2412.0, 1218.0, 0.0, 0.163, 0.106, 0.732], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.13, 0.87], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.235, 0.0, 0.765], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.267, 0.167, 0.567], [0.0, 0.0, 0.0, 0.0, 0.0, 0.084, 0.916], [0.0, 0.0, 0.0, 0.0, 0.12, 0.0, 0.88], [0.0, 0.0, 0.0, 0.0, 0.0, 0.137, 0.863], [6361.0, 355.0, 1334.0, 0.0, 0.245, 0.254, 0.501], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 5.0, 0.269, 0.0, 0.731], [0.0, 0.0, 0.0, 0.0, 0.0, 0.456, 0.544], [0.0, 0.0, 0.0, 0.0, 0.703, 0.0, 0.297], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.13, 0.87], [0.0, 0.0, 0.0, 0.0, 0.0, 0.295, 0.705], [0.0, 0.0, 0.0, 8.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 

In [ ]:
numerical_features_t = train[[ 'positive_emoji', 'negative_emoji', 'neutral_emoji', 'slang_rating','neg_score','pos_score','neu_score']]
print(numerical_features_t.values.tolist())

[[0.0, 0.0, 0.0, 0.0, 0.133, 0.0, 0.867], [1896.0, 2412.0, 1218.0, 0.0, 0.163, 0.106, 0.732], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.13, 0.87], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.235, 0.0, 0.765], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.267, 0.167, 0.567], [0.0, 0.0, 0.0, 0.0, 0.0, 0.084, 0.916], [0.0, 0.0, 0.0, 0.0, 0.12, 0.0, 0.88], [0.0, 0.0, 0.0, 0.0, 0.0, 0.137, 0.863], [6361.0, 355.0, 1334.0, 0.0, 0.245, 0.254, 0.501], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 5.0, 0.269, 0.0, 0.731], [0.0, 0.0, 0.0, 0.0, 0.0, 0.456, 0.544], [0.0, 0.0, 0.0, 0.0, 0.703, 0.0, 0.297], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.13, 0.87], [0.0, 0.0, 0.0, 0.0, 0.0, 0.295, 0.705], [0.0, 0.0, 0.0, 8.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 

In [ ]:
'''
Stacking numerical features along with textual features
'''

combinedFeatures = np.hstack([numerical_features_t, np.array(vector_matrix_x_train)])

numerical_features_test = test[[ 'positive_emoji', 'negative_emoji', 'neutral_emoji', 'slang_rating','neg_score','pos_score','neu_score']]
print(numerical_features_test.values.tolist())

combinedFeatures_test = np.hstack([numerical_features_test, np.array(vector_matrix_x_test)])

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.668, 0.332], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.255, 0.0, 0.745], [0.0, 0.0, 0.0, 10.0, 0.0, 0.146, 0.854], [0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.465, 0.535], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.085, 0.39, 0.525], [0.0, 0.0, 0.0, 0.0, 0.137, 0.0, 0.863], [6845.0, 3614.0, 4163.0, 0.0, 0.0, 0.687, 0.313], [2243.0, 189.0, 754.0, 0.0, 0.129, 0.495, 0.376], [0.0, 0.0, 0.0, 0.0, 0.0, 0.155, 0.845], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.108, 0.892], [0.0, 0.0, 0.0, 0.0, 0.0, 0.659, 0.341], [0.0, 0.0, 0.0, 0.0, 0.146, 0.0, 0.854], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.187, 0.0, 0.813], [0.0, 0.0, 0.0, 2.0, 0.559, 0.0, 0.441], [0.0, 0.0, 0.0, 0.0, 0.182, 0.0, 0.818], [0.0, 0.0, 0.0, 0.0, 0.304, 0.0, 0.696], [0.0, 0.0, 0.0, 0.0, 0.0, 0.291, 0.709], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.147, 0.0, 0.853], [0.0, 0.0, 0.0, 0.0

In [ ]:
from sklearn.model_selection import train_test_split
'''
Train test split
'''
x_train, x_valid, y_train, y_valid = train_test_split(combinedFeatures, y_train,test_size = 0.20, random_state = 42)

print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)
print(combinedFeatures_test.shape)

(9600, 57)
(2400, 57)
(9600,)
(2400,)
(4961, 57)


In [ ]:
x_train,y_train=combinedFeatures,y_train
x_val,y_val=combinedFeatures_test,y_val

In [ ]:
print(len(x_train),len(y_train),len(x_val),len(y_val))

12000 12000 4961 4961


In [ ]:
# standardization

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)
x_test = sc.transform(combinedFeatures_test)

In [ ]:
'''
SVM Classifier
'''

from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
model_svm = SVC()
model_svm.fit(x_train, y_train)

y_pred = model_svm.predict(x_valid)

print("Training Accuracy :", model_svm.score(x_train, y_train))
print("Validation Accuracy :", model_svm.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred,average='macro'))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)


Training Accuracy : 0.6983333333333334
Validation Accuracy : 0.57375
f1 score : 0.5779896040318199
[[484 205 235]
 [217 482  91]
 [238  37 411]]


In [ ]:
with open(r"drive/My Drive/NLP/glove_svm.pkl", "wb") as output_file:
    # pick_insert = open('drive/My Drive/NLP/data.pickle','wb')
    pickle.dump(model_svm, output_file)

In [ ]:
'''
Logistic Regression Model
'''

from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression()
model_lr.fit(x_train, y_train)

y_pred = model_lr.predict(x_valid)

print("Training Accuracy :", model_lr.score(x_train, y_train))
print("Validation Accuracy :", model_lr.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred,average='macro'))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)

Training Accuracy : 0.5652083333333333
Validation Accuracy : 0.5566666666666666
f1 score : 0.5606116103322153
[[456 219 249]
 [215 488  87]
 [248  46 392]]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
# with open(r"drive/My Drive/NLP/glove_lr.pkl", "wb") as output_file:
#     # pick_insert = open('drive/My Drive/NLP/data.pickle','wb')
#     pickle.dump(model_lr, output_file)

In [ ]:
'''
Decision Tree Classifier
'''

from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier()
model_dt.fit(x_train, y_train)

print(x_valid.shape)
y_pred = model_dt.predict(x_valid)

print("Training Accuracy :", model_dt.score(x_train, y_train))
print("Validation Accuracy :", model_dt.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred,average='macro'))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)

(2400, 57)
Training Accuracy : 0.9952083333333334
Validation Accuracy : 0.44375
f1 score : 0.4443511583507493
[[389 284 251]
 [264 384 142]
 [275 119 292]]


In [ ]:
# with open(r"drive/My Drive/NLP/glove_dt.pkl", "wb") as output_file:
#     # pick_insert = open('drive/My Drive/NLP/data.pickle','wb')
#     pickle.dump(model_dt, output_file)

In [ ]:
'''
xgBoost Classifier
'''

from xgboost import XGBClassifier

model_xg = XGBClassifier()
model_xg.fit(x_train, y_train)

y_pred = model_xg.predict(x_valid)

print("Training Accuracy :", model_xg.score(x_train, y_train))
print("Validation Accuracy :", model_xg.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred,average='macro'))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)

# pd.options.display.max_colwidth = 300
# train.head(2)
# t = train[1:]
# t.head()
# #print(test)
# print(y_valid)
# print(y_pred)

Training Accuracy : 0.6397916666666666
Validation Accuracy : 0.55
f1 score : 0.5530589437558747
[[428 228 268]
 [200 482 108]
 [216  60 410]]


In [ ]:
# with open(r"drive/My Drive/NLP/glove_xg.pkl", "wb") as output_file:
#     # pick_insert = open('drive/My Drive/NLP/data.pickle','wb')
#     pickle.dump(model_xg, output_file)

In [ ]:
'''
Random Forest Classifer
'''


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

model_rf = RandomForestClassifier()
model_rf.fit(x_train, y_train)

y_pred = model_rf.predict(x_valid)

print("Training Accuracy :", model_rf.score(x_train, y_train))
print("Validation Accuracy :", model_rf.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("F1 score :", f1_score(y_valid, y_pred,average='macro'))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)

Training Accuracy : 0.9952083333333334
Validation Accuracy : 0.5441666666666667
F1 score : 0.5480539270102011
[[457 216 251]
 [228 462 100]
 [249  50 387]]


In [ ]:
with open(r"drive/My Drive/NLP/glove_rf.pkl", "wb") as output_file:
    # pick_insert = open('drive/My Drive/NLP/data.pickle','wb')
    pickle.dump(model_rf, output_file)

##Demo

In [ ]:
def clean_tweet(tweet):
    tweet = re.sub('https.*$', '', tweet)  # remove URLs
    tweet = re.sub('RT|cc', '', tweet)  # remove RT and cc
    tweet = re.sub('#\S+', '', tweet)  # remove hashtags
    tweet = re.sub('@ \S+', '', tweet)  # remove mentions
    tweet = re.sub('[%s]' % re.escape("""!"#$%&'()*+-./:;<=>@[\]^_`{|}~"""), ' ', tweet)  # remove punctuations
    tweet = re.sub('\s+', ' ', tweet) 
    tweet = re.sub('[^a-zA-Z]', ' ', tweet) #remove characters other than alphabets
    tweet = tweet.lower()
    #tweet = word_tokenize(tweet) # remove repeated characters (hellooooo into hello)
    # remove extra whitespace
    return tweet

In [ ]:
!pip install emoji

In [ ]:
def extract_features(tweet):
  # 1. Emoji Analysis
  pos_emoji, neg_emoji, neu_emoji = find_emoji(tweet)
  
  # 2. Slang Existance and slang rating
  slang_exists, slang_rating = find_slang(tweet)

  # 3. Sentiment Analysis
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(tweet)
  neg_score = ss['neg']
  pos_score = ss['pos']
  neu_score = ss['neu']

  # 4. Clean data
  tweet0 = emoji.demojize(tweet)
  tweet1 = clean_tweet(tweet0)
  tweet2 = expand_sent(tweet1)
  review = tweet2.split(" ")
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  tweet3 = ' '.join(review)
  print(tweet3)
  print(len(train_corpus))
  train_corpus.append(tweet3)
  # # Bag of Words
  bag_of_words_vectorizer = CountVectorizer(max_features=1000)
  demo_sent=[]
  demo_sent.append(train_corpus[-1])
  print(len(demo_sent),type(demo_sent))
  print(len(train_corpus))
  X = bag_of_words_vectorizer.fit_transform(train_corpus)
  print("X shape",X.shape)
  words_counts = X.toarray()
  tfidf_transformer = TfidfTransformer()
  tfidf = tfidf_transformer.fit_transform(words_counts)
  a=tfidf[-1] 
  numerical_features_test = [neg_score, pos_score, neu_score, pos_emoji, neg_emoji, neu_emoji, slang_rating]
  # b=numerical_features_test.toarray()
  numerical_features_test = np.array(numerical_features_test)
  numerical_features_test = numerical_features_test.reshape((1,7))  
  print(numerical_features_test)
  F = np.hstack([numerical_features_test, a.toarray()])
  return F

In [ ]:
a=extract_features_glove("i am happy how are you")

happi 
12033
1 <class 'list'>
12034
X shape (12034, 1000)
[[0.    0.481 0.519 0.    0.    0.    0.   ]]


In [ ]:
import emoji

def extract_features_glove(tweet):
  # 1. Emoji Analysis
  pos_emoji, neg_emoji, neu_emoji = find_emoji(tweet)
  
  # 2. Slang Existance and slang rating
  slang_exists, slang_rating = find_slang(tweet)

  # 3. Sentiment Analysis
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(tweet)
  neg_score = ss['neg']
  pos_score = ss['pos']
  neu_score = ss['neu']

  # 4. Clean data
  tweet0 = emoji.demojize(tweet)
  tweet1 = clean_tweet(tweet0)
  tweet2 = expand_sent(tweet1)
  review = tweet2.split(" ")
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  tweet3 = ' '.join(review)
  
  train_corpus.append(tweet3)
  # # Bag of Words
  vector_matrix_x_demo=get_phrase_embedding(tweet3)
  print("glove embedding is",vector_matrix_x_demo)
  numerical_features_t = [neg_score, pos_score, neu_score, pos_emoji, neg_emoji, neu_emoji, slang_rating]
  F = np.hstack([numerical_features_t, np.array(vector_matrix_x_demo)])
  return F

In [ ]:
# with open(r"drive/My Drive/NLP/glove_svm.pkl", "rb") as input_file:
#     model_svm = pickle.load(input_file)
# with open(r"drive/My Drive/NLP/glove_dt.pkl", "rb") as input_file:
#     model_dt = pickle.load(input_file)
# with open(r"drive/My Drive/NLP/glove_lr.pkl", "rb") as input_file:
#     model_lr = pickle.load(input_file)
# with open(r"drive/My Drive/NLP/glove_xg.pkl", "rb") as input_file:
#     model_xg = pickle.load(input_file)
# with open(r"drive/My Drive/NLP/glove_rf.pkl", "rb") as input_file:
#     model_rf = pickle.load(input_file)

In [ ]:
with open(r"/data/trained_model_weights/Glove/glove_svm.pkl", "rb") as input_file:
    model_svm = pickle.load(input_file)
with open(r"/data/trained_model_weights/Glove/glove_dt.pkl", "rb") as input_file:
    model_dt = pickle.load(input_file)
with open(r"/data/trained_model_weights/Glove/glove_lr.pkl", "rb") as input_file:
    model_lr = pickle.load(input_file)
with open(r"/data/trained_model_weights/Glove/glove_xg.pkl", "rb") as input_file:
    model_xg = pickle.load(input_file)
with open(r"/data/trained_model_weights/Glove/glove_rf.pkl", "rb") as input_file:
    model_rf = pickle.load(input_file)

In [ ]:
def demo(sent):
  b=extract_features_glove(sent)
  b=b.reshape(1,57)
  y_dt=model_dt.predict(b)
  y_lr=model_lr.predict(b)
  y_rf=model_rf.predict(b)
  y_xg=model_xg.predict(b)
  y_svm=model_svm.predict(b)
  # print("Prediction by decision tree---",y_dt[0],"   ","Positive" if y_dt[0]==1 else "Neutral" if y_dt[0]==0 else "Negative" )
  # print("Prediction by Logistic Regression---",y_lr[0],"   ","Positive" if y_lr[0]==1 else "Neutral" if y_lr[0]==0 else "Negative" )
  # print("Prediction by Random Forest---",y_rf[0],"   ","Positive" if y_rf[0]==1 else "Neutral" if y_rf[0]==0 else "Negative")
  # print("Prediction by XG Boost---",y_xg[0],"   ","Positive" if y_xg[0]==1 else "Neutral" if y_xg[0]==0 else "Negative")
  print("Prediction Result---",y_svm[0],"   ","Positive" if y_svm[0]==1 else "Neutral" if y_svm[0]==0 else "Negative")
  

In [ ]:
def demo1(sent):
  b=extract_features_glove(sent)
  b=b.reshape(1,57)
  y_dt=model_dt.predict(b)
  y_lr=model_lr.predict(b)
  y_rf=model_rf.predict(b)
  y_xg=model_xg.predict(b)
  y_svm=model_svm.predict(b)
  # print("Prediction by decision tree---",y_dt[0],"   ","Positive" if y_dt[0]==1 else "Neutral" if y_dt[0]==0 else "Negative" )
  # print("Prediction by Logistic Regression---",y_lr[0],"   ","Positive" if y_lr[0]==1 else "Neutral" if y_lr[0]==0 else "Negative" )
  print("Prediction ---",y_rf[0],"   ","Positive" if y_rf[0]==1 else "Neutral" if y_rf[0]==0 else "Negative")
  # print("Prediction by XG Boost---",y_xg[0],"   ","Positive" if y_xg[0]==1 else "Neutral" if y_xg[0]==0 else "Negative")
  # print("Prediction Result---",y_svm[0],"   ","Positive" if y_svm[0]==1 else "Neutral" if y_svm[0]==0 else "Negative")

In [ ]:
demo("STOP watching their NONSENSE SHIT. It is NOT that important to your lives! NOR are so-called stars who become rich & famous at YOUR expense. &Why do people bother watching these कचरा? Lets save our youth &country")

glove embedding is [ 0.09391996  0.3086942  -0.14390446 -0.13736328 -0.06116056  0.04190289
  0.4722145   0.112189    0.1680341   0.11464628 -0.20457521 -0.04422851
 -3.0526218  -0.18254799 -0.09077528 -0.05433579 -0.04603336 -0.15578112
  0.01618565  0.2433303  -0.09431105 -0.23490402  0.09780223  0.15365198
 -0.31008172  0.37041324  0.28087583  0.13959008 -0.3822724  -0.10443395
  0.30612335 -0.06938072  0.11958778  0.22730476  0.19923462 -0.11165428
  0.08512309 -0.1733439   0.07018549 -0.37924606 -0.20104665  0.1949646
  0.26677138 -0.2441415   0.07506647  0.05809     0.3285288   0.18825044
 -0.07574928  0.17990834]
Prediction Result--- 0     Neutral


In [358]:
demo("@ruralIndiademands ,you modiji.chutiye hai ye congress wale")

glove embedding is [ 1.7581939e-03  3.3067203e-01  3.4263998e-01 -2.6279458e-01
 -2.3176181e-01  1.1621080e-01 -1.4288799e-01  6.3611805e-01
  5.0887994e-02  3.4949383e-01 -7.0285839e-01 -9.9234498e-01
 -2.0381401e+00  2.5960386e-02 -1.9622399e-01 -3.9599460e-01
 -1.1046000e-01 -2.7493602e-01  1.4067599e-01  8.7173998e-02
 -3.7906918e-01  2.4410605e-02 -6.3348401e-01  6.5975201e-01
 -2.9654402e-01 -1.4602998e-01  6.8794799e-01 -2.0194399e-01
 -1.7727540e-01  5.5550098e-01  9.4489205e-01  1.0866640e+00
  1.8265900e-01  1.0178600e-01  6.7597800e-01  1.5910201e-01
 -9.4372608e-02  5.6164157e-01  2.6588202e-01 -8.9165783e-01
 -4.3095002e-01 -1.4608781e-01 -3.6135921e-01  3.8072923e-01
 -6.0224599e-01  1.1978801e-01 -3.4999966e-03 -6.5767002e-01
  2.0205398e-01  1.5580200e-01]
Prediction Result--- 0     Neutral


In [366]:
demo("ye fraud modi maha chor hai pata nahi mere desh ki janta ki basi per inh vote kar")

glove embedding is [ 1.46697201e-02  2.42037117e-01 -1.83051489e-02 -5.20833194e-01
  5.64488843e-02 -2.17520464e-02 -5.38666427e-01  2.23236561e-01
  1.23947114e-01  4.71616060e-01 -6.49513364e-01 -7.33361125e-01
 -1.49494696e+00  5.69306090e-02 -9.06199738e-02  9.38351601e-02
  7.47966617e-02 -2.57337421e-01 -1.10546008e-01  1.91619679e-01
 -2.06808731e-01  9.57359895e-02 -4.33004260e-01  9.79567409e-01
 -3.70990157e-01 -4.96764451e-01  5.75518370e-01 -1.47059411e-01
 -1.05820544e-01  5.08433998e-01  9.13909078e-01  6.11027062e-01
  2.28514254e-01  6.25007823e-02  3.20514441e-01 -8.66749615e-04
 -2.31288642e-01  4.75306869e-01  1.58293340e-02 -7.21445858e-01
 -6.99541748e-01 -3.07069778e-01 -6.46735013e-01  2.93616176e-01
 -4.28122222e-01  3.15411150e-01 -1.24613248e-01 -2.99939096e-01
  3.86766702e-01  1.08976752e-01]
Prediction Result--- 2     Negative


In [371]:
demo("This week bahot dukh ka hafta hai")

glove embedding is [ 0.32448432 -0.09518666 -0.01494117 -0.5210677   0.08810073  0.03177832
 -0.36019835  0.05182    -0.32161465  0.5023767  -0.50556666 -0.5121367
 -1.374385    0.07273833 -0.670855   -0.0116535   0.04888166 -0.10319366
 -0.661141    0.36591175  0.112945    0.3943278  -0.72907835  0.8399157
 -0.7437784  -0.31606147  0.29224333 -0.20677532 -0.06071333  0.55001503
  0.9442999   0.67664176  0.53936833 -0.19644165  0.22474165 -0.13844812
 -0.09461335  0.17837     0.40053    -0.1879636  -0.99469995 -0.03909366
 -0.306245    0.7559984  -0.10607668  0.1969835  -0.02042834 -0.714735
  0.25992385  0.79861   ]
Prediction Result--- 1     Positive


In [367]:
demo("🙄🙄🙄🙄")

glove embedding is [-1.50629997e-01 -3.76993328e-01  9.25166607e-02  1.14633255e-02
  1.31078660e-01  3.69300060e-02  7.96433389e-01  4.26400006e-02
  7.78733417e-02  3.02656680e-01 -5.93850017e-02  8.00496638e-02
 -3.77853322e+00 -2.75404006e-01 -3.44010025e-01 -9.16900206e-03
 -3.49100046e-02 -3.60073298e-01  2.09999084e-03 -9.91209984e-01
 -6.72216713e-01  1.48526669e-01  4.48903352e-01 -1.28146708e-02
  1.05092324e-01 -1.84236690e-01 -3.70445341e-01  4.32190031e-01
 -3.39323312e-01  3.98490041e-01 -3.16186666e-01  2.15985000e-01
 -1.45279989e-01 -5.90020001e-01  1.25119984e-01  1.70413002e-01
 -1.87993441e-02 -4.94166613e-02 -1.59043327e-01  5.70133388e-01
 -8.29556763e-01 -1.00367337e-01 -1.17816664e-01  3.24726611e-01
  3.99143338e-01 -1.75400004e-01  6.32026613e-01  2.13479817e-01
  5.42210042e-01  2.78926641e-01]
Prediction Result--- 0     Neutral


In [365]:
demo("❤️❤️❤️❤️❤️❤️❤️")

glove embedding is [-8.29685092e-01 -7.84405053e-01  4.39899951e-01  4.94164944e-01
 -2.39905000e-01  1.30975977e-01  1.03526497e+00  1.62149996e-01
 -1.49549991e-02  1.10117495e-01  5.13604991e-02 -6.94999918e-02
 -4.14495039e+00 -9.94499996e-02 -1.82598755e-01  2.62804002e-01
  6.40925020e-02  1.31724998e-01  5.86805046e-01 -4.58230019e-01
 -6.31499946e-01  1.75125003e-01  1.22035019e-01 -1.47745013e-01
  4.65150058e-01 -2.43720025e-01 -7.20760047e-01  4.51940030e-01
  2.49109983e-01  1.36550069e-02 -2.95295000e-01 -8.50500241e-02
 -1.42799988e-02 -6.81049973e-02  6.15189970e-01 -1.18664511e-01
 -5.83350062e-02 -1.97479993e-01 -4.18285042e-01  5.31389952e-01
 -1.30154991e+00 -2.70494998e-01 -6.70109987e-02  6.58744946e-02
 -3.14000249e-03 -1.21466994e-01  5.67649961e-01 -2.65830010e-01
  2.52705008e-01  1.03096507e-01]
Prediction Result--- 1     Positive


In [ ]:
demo1("why don't these bastards just leave us in piece")

glove embedding is [-0.2008375   0.13998249  0.15704724  0.107461   -0.25934726 -0.5118
 -0.12385251 -0.19437751 -0.15917748  0.20579425  0.131395   -0.2634636
 -2.2850451  -0.30701226 -0.3320545   0.28236252  0.04681998 -0.17984
 -0.07484899  0.32986748 -0.10430248  0.09424065  0.2081175   0.2772445
 -0.06625451  0.67047     0.14828251 -0.08444724 -0.31677002 -0.008733
  0.7097      0.2579285   0.56716996  0.4123125  -0.0895475   0.1203135
 -0.003185    0.03148    -0.13442001 -0.534065    0.3050075   0.336975
  0.02285498 -0.13668999  0.024615    0.44253004  0.3869025   0.35294247
 -0.03614751  0.13986649]
Prediction --- 2     Negative


In [ ]:
demo("U saw caste religion nation saw talent trust problem tum paida hi ulte hue")

glove embedding is [ 0.26400262  0.16748118  0.11481928 -0.12036663 -0.10850443  0.1023062
  0.22948037  0.29962274 -0.07338122  0.31881016 -0.31130657 -0.14762302
 -2.8413284   0.17764156 -0.3671677   0.12001251 -0.04162472  0.06574751
 -0.05146857  0.17527999  0.130688    0.32603174 -0.12158829  0.29964072
 -0.32087785 -0.04260784  0.00932399 -0.07052529  0.03642849  0.24989474
  0.36702642  0.08218     0.13555558 -0.17559457  0.40184003  0.01614363
 -0.00994386  0.14118928 -0.02232886 -0.34538358 -0.45013556  0.02753251
 -0.05238285  0.1646348  -0.3511547   0.07875279  0.3673236  -0.32318947
  0.10021394 -0.10978659]
Prediction Result--- 0     Neutral


In [ ]:
demo("Ab gala faad ke nahi chillana chowkidaar chor hai Sharm nahi aayi aise naare lagate hue O")

glove embedding is [-1.63820148e-01  8.59031528e-02  1.70083880e-01 -8.96331131e-01
 -1.50145724e-01 -1.00316234e-01 -6.00242913e-01  3.48981433e-02
  1.49878144e-01  4.07086425e-02 -4.55579251e-01 -7.04039991e-01
 -9.84479904e-01  7.40565658e-02 -3.99609566e-01  3.16233307e-01
  1.42911702e-01 -1.60791159e-01 -2.23125711e-01  2.35130340e-01
  8.16290907e-04  2.51642853e-01 -2.33323067e-01  6.96572840e-01
 -2.83467382e-01 -6.09879017e-01  3.64808500e-01  1.98135199e-03
 -7.79122114e-02 -8.78463462e-02  9.81083333e-01  4.96514261e-01
  6.32409275e-01 -1.55231133e-01  4.81188625e-01 -4.15794939e-01
  4.24890630e-02  4.26422536e-01  4.28485870e-01 -6.61451459e-01
 -7.31464267e-01 -3.13143618e-02 -7.08116472e-01  5.50307989e-01
 -4.13751394e-01  3.35865259e-01 -1.77849025e-01 -3.60919029e-01
  4.90986407e-01  4.59096402e-01]
Prediction Result--- 0     Neutral
